In [1]:
import pandas as pd
import numpy as np
import scipy as sp


### 提取order的时间差值特征，以及最有的操作时间特征，
+ 和action进行相似的操作


In [2]:
df_train_data = pd.read_csv('../data/train/orderFuture_train.csv')
df_test_data = pd.read_csv('../data/test/orderFuture_test.csv')

df_train_order = pd.read_csv('../data/train/orderHistory_train.csv')
df_test_order = pd.read_csv('../data/test/orderHistory_test.csv')



In [3]:
# 最小的orderTime时间值，用来填充。
MINOREDETIME = 1478363873

MAXORDERTIME = 1505136953

DIFFORDERTIME = MAXORDERTIME - MINOREDETIME

In [4]:
df_train_order.head()



,userid,orderid,orderTime,orderType,city,country,continent
0,100000000013,1000015,1481714516,0,柏林,德国,欧洲
1,100000000013,1000014,1501959643,0,旧金山,美国,北美洲
2,100000000393,1000033,1499440296,0,巴黎,法国,欧洲
3,100000000459,1000036,1480601668,0,纽约,美国,北美洲
4,100000000459,1000034,1479146723,0,巴厘岛,印度尼西亚,亚洲


In [5]:
def diffInfo(x):
    if len(x) == 0:
        return x
    
    # 所有时间的最大值
    cur = MAXORDERTIME
    
    x = list(x)
    x.append(cur)
#     print(x)
    for i in range(len(x)-1):
        x[i] = x[i+1] - x[i]
    
    return np.array(x[:-1])



In [6]:
# [last1OrderTime, last1OrderTimeDiff, hasOrder, orderTypeContain1, 
#  lastOrderCity, lastOrderCountry, lastOrderContinent]
def getOrderTimeFeature(df_train_order):
    '''得到order的特征
    
    '''
    column = ['userid','last1OrderTime','last1OrderTimeDiff','OrderCnt','hasOrder','orderTypeContain1',
              'lastOrderCity','lastOrderCountry','lastOrderContinent']

    df_train_orderTime = pd.DataFrame(columns=column)

    userlist = list(set(df_train_order['userid']))
    k = 0
    for user in userlist:
        k+=1
        if k%1000 == 0:
            print(k)

        df_order = df_train_order[df_train_order['userid'] == user]
        df_order.sort_values('orderTime',inplace=True)

        df_order['orderTimeDiff'] = diffInfo(df_order['orderTime'])

        row = [user]

        # last1OrderTime
        if len(df_order) > 0:
            row.append(df_order['orderTime'].values[-1])
        else:
            row.append(MINOREDETIME)

        # last1OrderTimeDiff
        if len(df_order) > 0:
            row.append(df_order['orderTimeDiff'].values[-1])
        else:
            row.append(DIFFORDERTIME)
        
        # orderCnt
        row.append(len(df_order))
        
        # hasOrder
        row.append(len(df_order) > 0)

        # orderTypeContain1
        row.append(len(df_order[df_order['orderType']==1]) > 0)

        # 'lastOrderCity','lastOrderCountry','lastOrderContinent'
        if len(df_order) > 0:
            row.extend(df_order.values[-1][-4:-1])
        else:
            row.extend([np.nan,np.nan,np.nan])

        df_train_orderTime = df_train_orderTime.append(pd.DataFrame([row],columns=column),ignore_index=True)
    
    return df_train_orderTime


In [7]:
df_train_orderTime = getOrderTimeFeature(df_train_order)
df_test_orderTime = getOrderTimeFeature(df_test_order)


/home/xiaoran/Tools/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/xiaoran/Tools/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
1000
2000


In [8]:
df_train_orderTime.info()
df_test_orderTime.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10637 entries, 0 to 10636
Data columns (total 9 columns):
userid                10637 non-null object
last1OrderTime        10637 non-null object
last1OrderTimeDiff    10637 non-null object
OrderCnt              10637 non-null object
hasOrder              10637 non-null object
orderTypeContain1     10637 non-null object
lastOrderCity         10637 non-null object
lastOrderCountry      10637 non-null object
lastOrderContinent    10637 non-null object
dtypes: object(9)
memory usage: 748.0+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2686 entries, 0 to 2685
Data columns (total 9 columns):
userid                2686 non-null object
last1OrderTime        2686 non-null object
last1OrderTimeDiff    2686 non-null object
OrderCnt              2686 non-null object
hasOrder              2686 non-null object
orderTypeContain1     2686 non-null object
lastOrderCity         2686 non-null object
lastOrderCountry      2686 non-null object
last

In [9]:
df_train_data = pd.merge(df_train_data,df_train_orderTime,how='left',on='userid')
df_test_data = pd.merge(df_test_data,df_test_orderTime,how='left',on='userid')



In [15]:
df_train_data.info()
df_test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40307 entries, 0 to 40306
Data columns (total 9 columns):
userid                40307 non-null object
last1OrderTime        40307 non-null int64
last1OrderTimeDiff    40307 non-null int64
OrderCnt              40307 non-null int64
hasOrder              40307 non-null bool
orderTypeContain1     40307 non-null bool
lastOrderCity         40307 non-null int16
lastOrderCountry      40307 non-null int8
lastOrderContinent    40307 non-null int8
dtypes: bool(2), int16(1), int64(3), int8(2), object(1)
memory usage: 1.8+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 10076 entries, 0 to 10075
Data columns (total 9 columns):
userid                10076 non-null object
last1OrderTime        10076 non-null int64
last1OrderTimeDiff    10076 non-null int64
OrderCnt              10076 non-null int64
hasOrder              10076 non-null bool
orderTypeContain1     10076 non-null bool
lastOrderCity         10076 non-null int16
lastOrderCountry      

In [11]:
## 填充空值
df_train_data['last1OrderTime'].fillna(df_train_data['last1OrderTime'].min(),inplace=True)
df_train_data['last1OrderTimeDiff'].fillna(DIFFORDERTIME,inplace=True)

df_train_data['OrderCnt'].fillna(0,inplace=True)


df_train_data['hasOrder'].fillna(False,inplace=True)
df_train_data['orderTypeContain1'].fillna(False,inplace=True)

df_train_data['lastOrderCity'] = pd.Categorical(df_train_data['lastOrderCity']).codes
df_train_data['lastOrderCountry'] = pd.Categorical(df_train_data['lastOrderCountry']).codes
df_train_data['lastOrderContinent'] = pd.Categorical(df_train_data['lastOrderContinent']).codes


In [12]:
## 填充空值
df_test_data['last1OrderTime'].fillna(df_test_data['last1OrderTime'].min(),inplace=True)
df_test_data['last1OrderTimeDiff'].fillna(DIFFORDERTIME,inplace=True)

df_test_data['OrderCnt'].fillna(0,inplace=True)


df_test_data['hasOrder'].fillna(False,inplace=True)
df_test_data['orderTypeContain1'].fillna(False,inplace=True)

df_test_data['lastOrderCity'] = pd.Categorical(df_test_data['lastOrderCity']).codes
df_test_data['lastOrderCountry'] = pd.Categorical(df_test_data['lastOrderCountry']).codes
df_test_data['lastOrderContinent'] = pd.Categorical(df_test_data['lastOrderContinent']).codes


In [13]:
df_train_data['orderTypeContain1'].value_counts()

False    38562
True      1745
Name: orderTypeContain1, dtype: int64

In [14]:
df_train_data.drop('orderType',axis=1,inplace=True)
df_train_data.to_csv('../data/dataSet/df_train_5.csv',index=False,index_label=False)
df_test_data.to_csv('../data/dataSet/df_test_5.csv',index=False,index_label=False)
